In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

def page_source_for_soup(html): # 페이지 소스를 파싱해서 반환
    time.sleep(1)
    return BeautifulSoup(html,'html.parser')

In [6]:
# 1. 서울시 구별로 스타벅스 매장을 나누어 분석

driver = webdriver.Chrome() # 크롬 실행
time.sleep(1)
url = "https://www.starbucks.co.kr/store/store_map.do" # 접속할 페이지
driver.get(url) # 링크 접속
time.sleep(3) # 접속 시도 지연 시간

# 지역 검색 버튼 클릭
searchAreaBtn = "#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a"
driver.find_element("css selector", searchAreaBtn).click()
time.sleep(1)

# 서울 버튼 클릭
seoul_btn = "#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a"
driver.find_element("css selector", seoul_btn).click()
time.sleep(1)

# 전체 버튼 클릭
allAreaBtn = "#mCSB_2_container > ul > li:nth-child(1) > a"
driver.find_element("css selector", allAreaBtn).click()
time.sleep(1)

soup = page_source_for_soup(driver.page_source) # 현 페이지 내용 파싱
starbucksCount = soup.select("#mCSB_3_container > ul > li > p") # 서울시에 있는 모든 스타벅스 매장 주소 리스트

guDict = {} # 구: 매장 수 딕셔너리

for addr in starbucksCount:
    gu = re.findall("[가-힣]+구", addr.text.strip())[0] # 정규식을 이용하여 각 주소에서 구 이름만 추출합니다.
    if gu in guDict:
        guDict[gu] += 1 # guDict 딕셔너리에 같은 구 이름이 있으면 밸류를 1 증가 시킵니다.
    else: # 처음 등장한 구일 경우
        guDict[gu] = 1

maxArea = "" # 매장이 가장 많은 구
maxCount = 0 # 최대 매장 수
minArea = "" # 매장이 가장 적은 구
minCount = 0 # 최소 매장 수
allSum = 0 # 서울시 모든 매장 수 총 합

for k, v in guDict.items(): # 딕셔너리를 참조하여 매장이 가장 많은곳, 적은곳, 총 합을 계산합니다.
    if maxArea and minArea:
        if v > maxCount:
            maxArea = k
            maxCount = v
        elif v < minCount:
            minArea = k
            minCount = v
    else:
        maxArea = k
        maxCount = v
        minArea = k
        minCount = v
    allSum += v

print(maxArea, maxCount) # 가장 많은 구
print(minArea, minCount) # 가장 적은 구
print(int(allSum / len(guDict))) # 평균 매장 수

강남구 91
강북구 6
24


In [7]:
# 2. 인스타그램에서 '울릉도맛집'을 검색해서 나온 결과물을 스크래핑하여 텍스트만 추출

def insta_func(word): # 주소 고정 패턴 + word 반환
    return "https://www.instagram.com/explore/tags/" + word

driver = webdriver.Chrome() # 크롬 실행
time.sleep(1)
driver.get("https://www.instagram.com/") # 인스타 로그인 페이지 접속
time.sleep(1)
input_id = driver.find_element("css selector", "#loginForm > div > div:nth-child(1) > div > label > input") # 아이디 입력칸
input_pw = driver.find_element("css selector", "#loginForm > div > div:nth-child(2) > div > label > input") # 비번 입력칸
input_id.send_keys("horong0602@gmail.com") # 아이디 입력
time.sleep(1)
input_pw.send_keys("1qwe2asd3zxc@") # 비번 입력
time.sleep(1)
input_pw.submit() # 엔터
time.sleep(3) # 로그인 시도 지연 시간

word = "울릉도맛집" # 키워드
url = insta_func(word) # 접속될 링크
driver.get(url) # 접속
time.sleep(5) # 접속 시도 지연 시간

post = "div._aagw" # 첫번째 포스트 영역
driver.find_element("css selector", post).click() # 포스트 클릭
time.sleep(1)

for page in range(10):
    soup = page_source_for_soup(driver.page_source) # 현 페이지 내용 파싱

    postText = "div._a9zr" # 포스트 내용 영역
    print("<%d번째 게시물>" % (page + 1))
    print(soup.select(postText)[0].text) # 내용 출력
    
    # 댓글 텍스트 출력
    # 댓글 영역
    comments = "body > div.x1n2onr6.xzkaem6 > div.x9f619.x1n2onr6.x1ja2u2z > div > div.x1uvtmcs.x4k7w5x.x1h91t0o.x1beo9mf.xaigb6o.x12ejxvf.x3igimt.xarpa2k.xedcshv.x1lytzrv.x1t2pt76.x7ja8zs.x1n2onr6.x1qrby5j.x1jfb8zj > div > div > div > div > div.xb88tzc.xw2csxc.x1odjw0f.x5fp0pe.x1qjc9v5.xjbqb8w.x1lcm9me.x1yr5g0i.xrt01vj.x10y3i5r.xr1yuqi.xkrivgy.x4ii5y1.x1gryazu.x15h9jz8.x47corl.xh8yej3.xir0mxb.x1juhsu6 > div > article > div > div.x1qjc9v5.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x78zum5.xdt5ytf.x1iyjqo2.x5wqa0o.xln7xf2.xk390pu.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x65f84u.x1vq45kp.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1n2onr6.x11njtxf > div > div > div.x78zum5.xdt5ytf.x1q2y9iw.x1n2onr6.xh8yej3.x9f619.x1iyjqo2.x18l3tf1.x26u7qi.xy80clv.xexx8yu.x4uap5.x18d9i69.xkhd6sd > div.x78zum5.xdt5ytf.x1iyjqo2.xs83m0k.x2lwn1j.x1odjw0f.x1n2onr6.x9ek82g.x6ikm8r.xdj266r.x11i5rnm.x4ii5y1.x1mh8g0r.xexx8yu.x1pi30zi.x18d9i69.x1swvt13 > ul > div:nth-child(3) > div > div"
    print("\n<댓글>")
    print(soup.select(comments)[0].text) # 댓글 출력
    print("=" * 130)

    # 오른쪽 다음 버튼 클릭
    nextBtn = "body > div.x1n2onr6.xzkaem6 > div.x9f619.x1n2onr6.x1ja2u2z > div > div.x1uvtmcs.x4k7w5x.x1h91t0o.x1beo9mf.xaigb6o.x12ejxvf.x3igimt.xarpa2k.xedcshv.x1lytzrv.x1t2pt76.x7ja8zs.x1n2onr6.x1qrby5j.x1jfb8zj > div > div > div > div > div:nth-child(1) > div > div > div._aaqg._aaqh > button > div > span > svg"
    driver.find_element("css selector", nextBtn).click()
    time.sleep(1)

<1번째 게시물>
free__jenny내년 여름 울릉도 여행에 참고하시라고 만든 #울릉도맛집 모음Zip.🤤🔖 ‘신비섬횟집’ 물회 물회좀 먹어본 여자의 인생 물회 집..🥹 4박 5일 동안 두번 먹었대요. 먹는 방법이 특이해요. 비빔회로 한번, 물회로 두번 죠져 주는데 너무너무 맛있습니다!!! 🔖 ’현포교동반점‘ 중화 비빔밥, 탕수육 울릉도까지 와서 무슨 중식이냐😤 하시겠지만, 다이빙 끝나고 뜨끈한 온돌 방에서 먹는 중화비빔밥과 산더미 탕수육 정말 최고!!!! 짜장 짬뽕도 맛있었지만 중화비빔밥 진짜 불맛 최고😋🔖 ‘저동오징어회타운’ 강릉수산 울릉도에서 오징어🦑 수율이 급감해서.. 오징어 회는 다른 회와 같이 구매해야만 먹을수가 있는데요, 강릉 수산 정말 잘 챙겨주시고 횟감 상태들이 너무 싱싱 했어요. 마지막 사진은...헿.. #울릉도프리다이빙 해본 사람들은 다 알 사진🐚ㅋㅋ✅Tip. 울릉도 식당들은 네이버 지도에 나와있는 영업시간들과 다르게 운영될 수 있으니 전화 후 방문 필수📞‼️..#울릉도#프리다이빙#프리다이빙투어#프리다이버#커플취미#부부취미#부부여행#국내여행#울릉도캠핑#촌캉스#해루질#바다수영수정됨 · 22주

<댓글>
jmsyyh___35울릉도 한번 가봐야겠네요 좋네요22주좋아요 1개답글 달기댓글 옵션좋아요kyyjkh물회 특으로드셨군요 ㅋ 특 추천!22주답글 달기댓글 옵션좋아요답글 보기(1개)subin7444물회엔 소주🍶최애음식😍22주답글 달기댓글 옵션좋아요7246jjh우와 정말 음식들이 모두 맛있어보이네요.22주좋아요 1개답글 달기댓글 옵션좋아요jun060117👍👍👍🤩🤩🤩21주좋아요 1개답글 달기댓글 옵션좋아요giup0404골고루 드셨네20주답글 달기댓글 옵션좋아요95baekgun노을 경치가 운치있네요22주좋아요 1개답글 달기댓글 옵션좋아요skinscuba😊21주답글 달기댓글 옵션좋아요
<2번째 게시물>
jjungoo.world[#캄인블루프리다이빙] 제목 : 울릉도캠프 오세요~✨  캄인블루 울릉도캠프 모집 시작‼️ 캠프당 최대 인원 10명입니다 6월 캠프 

In [ ]:
driver = webdriver.Chrome() # 크롬 실행
time.sleep(1)
url = "https://game.naver.com/esports/League_of_Legends/schedule/lck?date=2024-03" # 접속할 페이지

driver.get(url) # 링크 접속
time.sleep(3) # 접속 시도 지연 시간

soup = page_source_for_soup(driver.page_source) # 현 페이지 내용 파싱
for i in range(2, 30):
    try:
        playResult = "#civ > div > div > div > div.schedule_container__2rbMY > div:nth-child(2) > div.list_wrap__3zIxG > div:nth-child(%d)" % (i)
        playResult = soup.select(playResult)[0].text.strip()
    except:
        break
    else:
        date = re.findall("[0-9월일 ]+[(][가-힣]{1}[)]", playResult)
        time = re.findall("[0-9]{2}:[0-9]{2}", playResult)
        print(date)
        print(time)
        print("\n")